In [124]:
import pandas as pd
from get_results import *
import pickle
from datetime import date
import os
cwd = os.getcwd()
import pdb
import numpy as np
import random

### get_results()

In [2]:
import requests
import json
import pickle
import os
import pdb
cwd = os.getcwd()

def process_match(match):
    hometeam = match["homeTeam"]["name"]
    awayteam = match["awayTeam"]["name"]
    homescore = match["score"]["fullTime"]["home"]
    awayscore = match["score"]["fullTime"]["away"]
    group =  match["group"]
    stage = match["stage"]
    score = f"{homescore} - {awayscore}"
    if stage == "GROUP_STAGE":
        letter = group[-1]
        matchid = f"Group {letter} Predictions [{hometeam} - {awayteam}]"
    else:
        matchid = f"{stage} Predictions [{hometeam} - {awayteam}]"
    return matchid,score

def get_results():
    uri = 'https://api.football-data.org/v4/competitions/EC/matches'
    headers = { 'X-Auth-Token': '242e02ff31ea497fbe4b85978fe70b81' }
    
    response = requests.get(uri, headers=headers)
    
    matches  = response.json()["matches"]
    results = []
    for m in matches:
        results.append(process_match(m))
    return results


def save_results(filename,a):
    with open(filename, 'wb') as handle:
        pickle.dump(a, handle)

def load_results(filename):
    with open(filename, 'rb') as handle:
        b = pickle.load(handle)
    return b

# Function that fetches the largest index of downloaded data
def get_highest_result_number():
    n_file = 0
    for i in range(100):
        isf = os.path.isfile(cwd + f"/results/data_{i}.pickle")
        if isf:
            n_file = i
    return n_file

### main

In [3]:
results = get_results()
date = date.today()
datafile = [results,date]
n_file = get_highest_result_number()

prev_results = load_results(cwd + f"/results/data_{n_file}.pickle")
if prev_results[0] != results:
    print("New results saved")
    save_results(cwd + f"/results/data_{n_file+1}.pickle",datafile)


predictions = pd.read_csv("EM spillet 2024.csv")

In [16]:
# Eval predictions against results - DONE

def eval_match_predictions(predictions_df , results):
    # Convert results to a dict (and select only group stage matches)
    results_dict = {results[x][0]:results[x][1] for x in range(len(results)) if "Group" in results[x][0]}
    # Select the group stage matches in predictions
    predictions = predictions_df.iloc[:,3:39]
    
    # Initialize empty points var
    points = 0
    for i in range(len(predictions)):
        pred = predictions.iloc[0,i]
        res = results_dict[predictions.columns[i]]
        
        # Check for nan (to be able to do temporary standings)
        if res != res or pred != pred:
            continue
        else:
            ### 15 points, correct score for both teams
            if pred == res:
                points += 15
        
            ### 10 points, correct outcome and score for one team
            # Home team wins and correct score of home team    
            elif res[0]>res[2] and pred[0]>pred[2] and res[0] == pred[0]:
                points += 10
            # Home team wins and correct score of away team  
            elif res[0]>res[2] and pred[0]>pred[2] and res[2] == pred[2]:
                points += 10
            # Away team wins and correct score of home team  
            elif res[0]<res[2] and pred[0]<pred[2] and res[0] == pred[0]:
                points += 10
            # Away team wins and correct score of away team  
            elif res[0]<res[2] and pred[0]<pred[2] and res[2] == pred[2]:
                points += 10
            
            ### 5 points, correct outcome (winner or tie)
            # Home team wins
            elif res[0]>res[2] and pred[0]>pred[2]:
                points += 5
            # Away team wins
            elif res[0]<res[2] and pred[0]<pred[2]:
                points += 5
            # Tie
            elif res[0]==res[2] and pred[0]==pred[2]:
                points += 5   
                
            ### 2 points, correct score for one team
            # Home team correct score
            elif res[0]==pred[0]:
                points += 2
            # Away team correct score
            elif res[2]==pred[2]:
                points += 2
                
    return points

'2-0'

In [65]:
results_dict = {results[x][0]:results[x][1] for x in range(len(results)) if "Group" in results[x][0]}
results_dict

{'Group A Predictions [Germany - Scotland]': 'None - None',
 'Group A Predictions [Hungary - Switzerland]': 'None - None',
 'Group B Predictions [Spain - Croatia]': 'None - None',
 'Group B Predictions [Italy - Albania]': 'None - None',
 'Group D Predictions [Poland - Netherlands]': 'None - None',
 'Group C Predictions [Slovenia - Denmark]': 'None - None',
 'Group C Predictions [Serbia - England]': 'None - None',
 'Group E Predictions [Romania - Ukraine]': 'None - None',
 'Group E Predictions [Belgium - Slovakia]': 'None - None',
 'Group D Predictions [Austria - France]': 'None - None',
 'Group F Predictions [Turkey - Georgia]': 'None - None',
 'Group F Predictions [Portugal - Czechia]': 'None - None',
 'Group B Predictions [Croatia - Albania]': 'None - None',
 'Group A Predictions [Germany - Hungary]': 'None - None',
 'Group A Predictions [Scotland - Switzerland]': 'None - None',
 'Group C Predictions [Slovenia - Serbia]': 'None - None',
 'Group C Predictions [Denmark - England]': 'No

In [126]:
results_dict2 = results_dict.copy()
results_dict3 = {k:str(random.randint(0,4))+"-"+str(random.randint(0,4)) for k,v in results_dict2.items()}

In [157]:
all_group_names = np.unique([k[:7] for k,v in results_dict3.items()]).tolist()
all_group_res = {group:{"1st":"","2nd":""} for group in all_group_names}


for group_name in all_group_names:
    group_results = {k: v for k,v in results_dict3.items() if k[:7] == group_name}
    group_home_countries = [k.split("[")[-1].split("]")[0].split("-")[0].strip() for k in list(group_results.keys())]
    group_away_countries = [k.split("[")[-1].split("]")[0].split("-")[1].strip() for k in list(group_results.keys())]
    group_countries = np.unique(group_home_countries + group_away_countries).tolist()
    
    group_eval = {country:{"points":0,"goals_for":0,"goals_against":0}  for country in group_countries}
    
    for k,v in group_results.items():
        home_team = k.split("[")[-1].split("]")[0].split("-")[0].strip()
        away_team = k.split("[")[-1].split("]")[0].split("-")[1].strip()    
        
        home_score = v.split("-")[0].strip()
        away_score = v.split("-")[1].strip()
        
        if home_score == "None" or away_score == "None":
            print("No results for",home_team,"-",away_team)

        else:
            group_eval[home_team]["goals_for"] += int(home_score)
            group_eval[home_team]["goals_against"] += int(away_score)
            
            group_eval[away_team]["goals_for"] += int(away_score)
            group_eval[away_team]["goals_against"] += int(home_score)
            
            if int(home_score) > int(away_score):
                group_eval[home_team]["points"] += 3
            elif int(home_score) < int(away_score):
                group_eval[away_team]["points"] += 3
            elif int(home_score) == int(away_score):
                group_eval[away_team]["points"] += 1
                group_eval[home_team]["points"] += 1
                                
    # Find 1st and 2nd place in group
    group_stand = {k:v["points"] for k,v in group_eval.items()}
    group_stand = dict(sorted(group_stand.items(), key=lambda item: item[1], reverse = True))
     
    if sum(np.array(list(group_stand.values())) == max(list(group_stand.values()))) == 1:
        # The scenario where only 1 team has max points
        all_group_res[group_name]["1st"] = list(group_stand.keys())[0]
        # Remove 1st team
        del group_stand[list(group_stand.keys())[0]]
        
        # Find 2nd best team
        if sum(np.array(list(group_stand.values())) == max(list(group_stand.values()))) == 1:
            # The scenario where only 1 team has 2nd most points
            all_group_res[group_name]["2nd"] = list(group_stand.keys())[0]
        elif sum(np.array(list(group_stand.values())) == max(list(group_stand.values()))) > 1:
            # The scenario where multiple teams have 2nd most points - look into match results
            
            # The scenario where multiple teams have max points - look into goal difference
            equal_teams = np.array(list(group_stand.keys()))[np.array(list(group_stand.values())) == max(list(group_stand.values()))].tolist()
            equal_points = {team:group_eval[team]["goals_for"]-group_eval[team]["goals_against"] for team in equal_teams}
            equal_points = dict(sorted(equal_points.items(), key=lambda item: item[1], reverse = True))
            all_group_res[group_name]["2nd"] = list(equal_points.keys())[0]
            
    elif sum(np.array(list(group_stand.values())) == max(list(group_stand.values()))) > 1:
        # The scenario where multiple teams have max points - look into match results
        
        # The scenario where multiple teams have max points - look into goal difference
        
        # Find the equal teams
        equal_teams = np.array(list(group_stand.keys()))[np.array(list(group_stand.values())) == max(list(group_stand.values()))].tolist()
        # Compute goal difference for each equal team 
        equal_points = {team:group_eval[team]["goals_for"]-group_eval[team]["goals_against"] for team in equal_teams}
        # Sort the equal teams based on goal difference 
        equal_points = dict(sorted(equal_points.items(), key=lambda item: item[1], reverse = True))
        
        # Check if goal difference is equal
        
        # Select winners
        all_group_res[group_name]["1st"] = list(equal_points.keys())[0]
        all_group_res[group_name]["2nd"] = list(equal_points.keys())[1]
        
        # There is an unlikely edge case: equal teams can have same goal difference. Then it is down to most goal scored. 
        # If that is also equal then it is down to the score of the match between the teams.
    
    print(group_eval)    
    

{'Germany': {'points': 4, 'goals_for': 7, 'goals_against': 8}, 'Hungary': {'points': 3, 'goals_for': 5, 'goals_against': 5}, 'Scotland': {'points': 5, 'goals_for': 8, 'goals_against': 6}, 'Switzerland': {'points': 4, 'goals_for': 5, 'goals_against': 6}}
{'Albania': {'points': 4, 'goals_for': 11, 'goals_against': 11}, 'Croatia': {'points': 7, 'goals_for': 11, 'goals_against': 9}, 'Italy': {'points': 3, 'goals_for': 9, 'goals_against': 9}, 'Spain': {'points': 3, 'goals_for': 7, 'goals_against': 9}}
{'Denmark': {'points': 7, 'goals_for': 10, 'goals_against': 8}, 'England': {'points': 5, 'goals_for': 5, 'goals_against': 4}, 'Serbia': {'points': 2, 'goals_for': 6, 'goals_against': 7}, 'Slovenia': {'points': 1, 'goals_for': 4, 'goals_against': 6}}
{'Austria': {'points': 6, 'goals_for': 10, 'goals_against': 4}, 'France': {'points': 3, 'goals_for': 4, 'goals_against': 6}, 'Netherlands': {'points': 3, 'goals_for': 3, 'goals_against': 6}, 'Poland': {'points': 6, 'goals_for': 3, 'goals_against': 

In [158]:
all_group_res

{'Group A': {'1st': 'Scotland', '2nd': 'Germany'},
 'Group B': {'1st': 'Croatia', '2nd': 'Albania'},
 'Group C': {'1st': 'Denmark', '2nd': 'England'},
 'Group D': {'1st': 'Austria', '2nd': 'Poland'},
 'Group E': {'1st': 'Belgium', '2nd': 'Romania'},
 'Group F': {'1st': 'Portugal', '2nd': 'Turkey'}}

In [ ]:
# Based on results, find group winners
def eval_group_winners(results):
    results_dict = {results[x][0]:results[x][1] for x in range(len(results)) if "Group" in results[x][0]}
    


## Status
- Continue developing logic determing group winners (inbyrdes kampe,målforskel,scorede mål)
- test logic
- Put everythin into get results